In [1]:
import urllib
import requests
from bs4 import BeautifulSoup
from functools import cache

In [2]:
from openai import OpenAI
client = OpenAI(api_key="")

In [3]:
@cache
def send_request(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    return response

In [4]:
import re

def extract_links(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    a_tags = soup.find_all('a', recursive=True)
    for a in a_tags:
        h3_tags = a.find_all('h3', recursive=True)
        for _ in h3_tags:
            href = a['href']
            match = re.search(r'url=(.*?)&', href)
            if match:
                yield match.group(1)

In [5]:
def search_google(query):
    encoded_query = urllib.parse.quote_plus(query)
    url = f"https://www.google.com/search?q={encoded_query}"
    response = send_request(url)
    return list(extract_links(response.text))

In [6]:
def extract_body_content(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the body tag
    body = soup.find("body")

    if body:
        # Remove unwanted tags
        for element in body.find_all(["script", "style", "meta", "noscript"]):
            element.decompose()

        return str(body)

In [7]:
from bs4 import BeautifulSoup
import re

def clean_html(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all elements
    for tag in soup.find_all():
        # if tag.name == 'a':
        #     # If it's an 'a' tag, keep only the href attribute
        #     href = tag.get('href')
        #     tag.attrs = {}  # Clear all attributes
        #     if href:
        #         tag['href'] = href  # Add back the href if it existed
        # else:
        # If it's not an 'a' tag, replace it with its contents
        tag.replace_with_children()

    # Return the modified HTML as a string
    text_content = str(soup)
    return re.sub(r'\s+', ' ', text_content)

In [8]:
query = "Earth is disovered to be flat"
links = search_google(query)


In [9]:
from markdownify import markdownify as md

def prepare_context(links):
    context = ""
    for link in links:
        try:
            response = send_request(link)
            body = extract_body_content(response.text)
            content = clean_html(body)
            markdown_content = md(content)
            top_1000_words = " ".join(markdown_content.split()[:1000])
            context += top_1000_words + "\n\n"
        except Exception as e:
            print(f"Error processing link: {link}. Error: {e}")
    return context
context = prepare_context(links)

In [14]:
prompt = """
I give you an information body from news and given a prompt you should detect if the prompt is a fake news or not briefly. I want to make decision accordingly. Just tell me if it's fake or not. Do not add anything else. Here is the prompt:
Prompt:
{query}
Information body:
{context}
""".format(context=context, query=query)

In [15]:
def call_llm(prompt, model="gpt-4o-mini"):
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    )
    return response.choices[0].message.content

In [16]:
call_llm(prompt)

'Fake.'